## Add TIRCP

In [2]:
import _utils
import numpy as np
import pandas as pd
from calitp.sql import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
pd.options.display.max_columns = 200
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
# import fuzzywuzzy
# from fuzzywuzzy import process

### ATP - 10 Year Non SHOPP Prep

In [5]:
# Read in 10 Year non SHOPP
atp_shopp = to_snakecase(
    pd.read_excel(f"{_utils.GCS_FILE_PATH}cleaned_data_with_fake_metrics_plus_atp.xlsx")
)

In [6]:
# list(atp_shopp.columns)

In [7]:
# Subset
atp_shopp_subset = [
    "ppno",
    "project_name",
    "lead_agency",
    "previous_caltrans_nominations",
    "full_county_name",
    "district",
    "project_description",
    "current_phase",
    "primary_mode",
    "urban_rural",
    "total_project_cost__$1,000",
    "total_unfunded_need__$1,000",
    "notes",
    "shs_capacity_increase_detail",
    "current_phase",
]

In [8]:
# Column for testing stuff
atp_shopp2 = atp_shopp[atp_shopp_subset]

In [9]:
# atp_shopp2.sample()

In [10]:
# Function to clean agency/organization names
def organization_cleaning(df, column_wanted: str):
    df[column_wanted] = (
        df[column_wanted]
        .str.strip()
        .str.split(",")
        .str[0]
        .str.replace("/", "")
        .str.split("(")
        .str[0]
        .str.split("/")
        .str[0]
        .str.title()
        .str.replace("Trasit", "Transit")
        .str.strip()  # strip again after getting rid of certain things
    )
    return df

In [11]:
# Lowercase previous caltrans nominations
atp_shopp2.previous_caltrans_nominations = (
    atp_shopp2.previous_caltrans_nominations.str.lower()
)

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
# Clean ATP
atp_shopp2 = organization_cleaning(atp_shopp2, "lead_agency")

/tmp/ipykernel_1422/2101408362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# atp_shopp2.head(100)

### TIRCP Prep
* Filter out projects in which total cost equals total award 

In [14]:
# Read in TIRCP. Last updated November 2022.
tircp = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/tircp/Tableau_Workbook.xlsx"
    )
)

In [15]:
# tircp.columns

In [16]:
# Subset TIRCP with only the basic information
tircp_subset = [
    "award_year",
    "grant_recipient",
    "title",
    "ppno",
    "district",
    "county",
    "description",
    "total__cost",
    "tircp",
    "award_cycle",
    "on_shs?",
    "comments_additional_contacts",
]

In [17]:
tircp2 = tircp[tircp_subset]

In [18]:
# Create a column with cycle + tircp for previous CT nominations
tircp2["previous_caltrans_nominations"] = (
    "TIRCP" + " Cycle " + tircp2["award_cycle"].astype("str")
)

/tmp/ipykernel_1422/2189071234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
# Create column for unmet needs
tircp2["total_unfunded_need__$1,000"] = tircp2["total__cost"] - tircp2["tircp"]

/tmp/ipykernel_1422/2212343423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
# Perhaps to narrow down projects
# Figure out which TIRCP projects' total cost are completely covered by TIRCP requested
tircp2["total_cost_vs_tircp_req"] = tircp2["tircp"] / tircp2["total__cost"]

/tmp/ipykernel_1422/2822305151.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
# tircp2.total_cost_vs_tircp_req.value_counts()

In [22]:
# Filter out projects in which total cost equals total award
tircp2 = (tircp2.loc[tircp2.total_cost_vs_tircp_req != 1]).reset_index(drop=True)

In [23]:
# Drop some columns
tircp_cols_drop = ["tircp", "total_cost_vs_tircp_req", "award_cycle"]

In [24]:
tircp2 = tircp2.drop(columns=tircp_cols_drop)

In [25]:
tircp2.shape

(93, 12)

#### Get current phase & amounts.

In [26]:
allocation = to_snakecase(
    pd.read_excel(
        f"gs://calitp-analytics-data/data-analyses/tircp/clean_tircp.xlsx",
        sheet_name="clean_allocation",
    )
)

/tmp/ipykernel_1422/3824326962.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})


In [27]:
# allocation.isna().sum()

In [28]:
# Subset - date is allocation date and has the least # of missing values in the date field
allocation2 = allocation[["award_year", "ppno", "phase", "date"]]

In [29]:
# Find the most recent allocation date & phase.
allocation3 = (
    allocation2.groupby(["award_year", "ppno", "phase"]).agg({"date": "max"})
).reset_index()

In [30]:
# Only keep the most recent allocation date & phase.
allocation3 = allocation3.sort_values(
    ["ppno", "date"], ascending=[True, False]
).drop_duplicates("ppno")

In [31]:
# Drop date
allocation3 = allocation3.drop(columns=["date"])

In [32]:
# Merge
tircp2 = pd.merge(
    tircp2,
    allocation3,
    how="left",
    on=["ppno", "award_year"],
)

In [33]:
len(tircp2)

93

#### Harmonize the way columns are named/formatted.
* TIRCP: `Total_cost` and `unmet needs` should be floats and divided by 1000. `District` should be int64. 

In [34]:
atp_shopp2.previous_caltrans_nominations.nunique(), len(atp_shopp2)

(242, 814)

In [35]:
# Find projects that have DRMT in previous caltrans nominations
# The CT nominations column has been casted to lower case. 
atp_shopp_drmt = (
    atp_shopp2[atp_shopp2["previous_caltrans_nominations"].str.contains(("tircp|drmt"))]
).reset_index(drop=True)

In [36]:
# Check out that the filtering was correct.
# atp_shopp_drmt.previous_caltrans_nominations.value_counts()

In [37]:
# Natalie's function
def align_funding_numbers(df, list_of_cols):
    for col in list_of_cols:
        df[col] = df[col] / 1000

    return df

In [38]:
tircp2 = align_funding_numbers(
    tircp2,
    [
        "total_unfunded_need__$1,000",
        "total__cost",
    ],
)

In [39]:
# (tircp2[["total_unfunded_need__$1,000"]]/1000).astype('int64')*1000

In [40]:
tircp2[["total_unfunded_need__$1,000", "total__cost",]] = tircp2[
    [
        "total_unfunded_need__$1,000",
        "total__cost",
    ]
].fillna(0)

In [41]:
tircp2["district_full_name"] = tircp2["district"]

In [42]:
# Extract digit from district. Fill "Various" districts with "0" and change to int64
tircp2["district"] = tircp2["district"].str.extract("(\d+)").fillna(0).astype("int64")

In [43]:
# Rename columns
tircp_new_cols = {
    "grant_recipient": "lead_agency",
    "title": "project_name",
    "county": "full_county_name",
    "description": "project_description",
    "total__cost": "total_project_cost__$1,000",
    "on_shs?": "shs_capacity_increase_detail",
    "comments_additional_contacts": "notes",
    "phase": "current_phase",
}

In [44]:
# atp_shopp_drmt.info(), tircp2.info()

In [45]:
tircp2 = tircp2.rename(columns=tircp_new_cols)

In [46]:
# Drop columns
tircp2 = tircp2.drop(columns=["award_year"])

In [47]:
# Add column for primary mode.
tircp2["primary_mode"] = "Transit/Zev/Rail (Passenger)"

In [48]:
tircp2.columns

Index(['lead_agency', 'project_name', 'ppno', 'district', 'full_county_name',
       'project_description', 'total_project_cost__$1,000',
       'shs_capacity_increase_detail', 'notes',
       'previous_caltrans_nominations', 'total_unfunded_need__$1,000',
       'current_phase', 'district_full_name', 'primary_mode'],
      dtype='object')

In [49]:
district_replace = {
    "District 7: Los Angeles": "07 - Los Angeles",
    "District 4: Bay Area / Oakland": "04 - Oakland",
    "District 5: San Luis Obispo / Santa Barbara": "05 - San Luis Obispo",
    "District 12: Orange County": "12 - Irvine",
    "District 3: Marysville / Sacramento": "03 - Marysville",
    "District 11: San Diego": "11 - San Diego",
    "District 10: Stockton": "10 - Stockton",
    "District 6: Fresno / Bakersfield": "06 - Fresno",
    "District 8: San Bernardino / Riverside": "08 - San Bernardino",
    "District 2:Redding": "02 - Redding",
    "District 1: Eureka": "01 - Eureka",
}

In [50]:
# Replace district name to match
tircp2.district = tircp2.district.replace(district_replace)

### Find TIRCP projects that are already in the 10 year Non SHOPP. 

#### Test with Merges
* Zero merges across the board. 

In [51]:
# Make a copy of tircp
# tircp_test = tircp2.copy()

In [52]:
# Round numbers
# tircp_test["total_unfunded_need__$1,000"] = (tircp_test[["total_unfunded_need__$1,000"]]/1000).astype('int64')*1000

In [53]:
# tircp_test["total_project_cost__$1,000"] = (tircp_test[["total_project_cost__$1,000"]]/1000).astype('int64')*1000

In [54]:
# atp_shopp_drmt["total_project_cost__$1,000"] = ((atp_shopp_drmt[["total_project_cost__$1,000"]]/1000).astype('int64')*1000).astype('int64')
# atp_shopp_drmt["total_project_cost__$1,000"] = ((atp_shopp_drmt[["total_project_cost__$1,000"]]/1000).astype('int64')*1000).astype('int64')

In [55]:
# Merge on district and unfunded needs
# test1 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", on=["district", "total_unfunded_need__$1,000"], indicator = True)

In [56]:
# test1._merge.value_counts()

In [57]:
# Merge on county and unfunded needs
# test2 = pd.merge(atp_shopp_drmt, tircp2, how= "left", on=["full_county_name", "total_unfunded_need__$1,000"], indicator = True, suffixes = ["_shopp", "_tircp"])

In [58]:
# test2._merge.value_counts()

In [59]:
# test2.loc[test2._merge == "both"][["project_name_shopp","project_name_tircp", "full_county_name", "project_description_shopp","project_description_tircp"]]

In [60]:
# Merge on district and total costs
# test2 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", on=["district", "total_project_cost__$1,000"], indicator = True)

#### Observation: 
* Total Project Cost, project titles, lead agency, and districts are potentially listed differently across datasets.
* Inglewood Transit Connector Project	  is listed as $1,016,000.000 in TIRCP but 1,666,466.0 in non SHOPP.
    * City Of Inglewood is the grant recipient in TIRCP but Caltrans is the lead agency in Non SHOPP.
* Valley Rail Expansion: Altamont Corridor Express (ACE) Ceres to Turlock Extension (the TIRCP title) is listed as D6 in TIRCP but D10 in Stanislaus.
    * Fresno Subdivision (Ceres To Turlock) Double Tracking is the non SHOPP title.

In [61]:
tircp_already_entered = [
    "Inglewood Transit Connector Project",
    "Valley Rail Expansion: Altamont Corridor Express (ACE) Ceres to Turlock Extension",
]

In [120]:
tircp.loc[tircp.title.isin(tircp_already_entered)][['district','description']]

,district,description
59,District 7: Los Angeles,"Construction of a 1.6-mile electrically powered automated people mover (APM) system and three new stations in the City of Inglewood. The project will create a new connection for passengers directly from the LA Metro Crenshaw/LAX Line’s Downtown Inglewood Station to new housing and employment centers, and regionally serving sports and entertainment including the Los Angeles Sports and Entertainment District (LASED) at Hollywood Park/SoFi Stadium and the proposed Inglewood Basketball and Entertainment Center (IBEC) Project. The project will connect the City of Inglewood’s high growth areas with LA Metro’s regional rail system."
91,District 6: Fresno / Bakersfield,"Extends ACE from Ceres to Turlock, which is an interim phase of the Ceres to Merced extension. Includes a new Turlock Station and layover track, and provides a direct connection with Turlock Transit"


In [121]:
# Projects with only TIRCP in the previous nomination.
atp_shopp_drmt[atp_shopp_drmt["previous_caltrans_nominations"].str.contains(("tircp"))][['project_name','district','project_description','primary_mode','previous_caltrans_nominations']]

,project_name,district,project_description,primary_mode,previous_caltrans_nominations
4,"Los Angeles Metro Light Rail Capital, Operational And Rehabilitation Enhancements (Core) Capacity & System Integration Project",7,"The project will eliminate a two-car capacity constraint on the integrated Crenshaw/LAX and Green (C) light rail lines, which operate within the congested I-405 and I-105 corridors and the area around Los Angeles International Airport (LAX), by:\n1) Extending the aerial platforms at four Green Line stations (Redondo Beach, Douglas, Mariposa, and Aviation/LAX)\n2) State of good repair work and station improvements at all of those four stations plus El Segundo Station\n3) Adding two new traction power substations (TPSS) on the Crenshaw/LAX Line in the cities of Inglewood and Los Angeles",Rail (Passenger),sccp cycle 3 priority 13 of 14\nnot awarded tircp
6,Inglewood Transit Connector (Itc) Project,7,"The Project consists of an approximately 1.6-mile fully elevated transit system in the City of Inglewood with three transit station connecting passengers from the Metro Crenshaw/LAX (K) Line to the City's new housing and employment centers and sports and entertainment venues. The Project is primarily located along Market Street, Manchester Boulevard and Prairie Avenue and includes a fully elevated guideway and supporting infrastructure, automated trains, a maintenance and storage facility, power distribution substations, and new public parking lots.",Transit,tircp award was downscaled\nsccp cycle 3 priority 05 of 14\nawarded raise 2022 (not caltrans)
7,Fresno Subdivision (Ceres To Turlock) Double Tracking,10,"Construct double tracking of 8.1 miles of existing single-track UPRR mainline rail corridor to alleviate congestion, increase capacity, improve safety, and allow for more efficient freight rail movement. A secondary benefit is the project is also necessary for the extension of Altamont Corridor Express (ACE) passenger rail services from Ceres to Turlock.",Rail (Freight),tircp 2021 application - pending award\ninfra 2022 application - pending award\ntcep cycle 3 priority 04 of 24\nmpdg los 2022 signed


In [63]:
# Delete TIRCP projects that are already in non SHOPP
# One project below in non SHOPP mentions TIRCP but that project
# was ultimately not nominated.
tircp2 = (tircp2[~tircp2["project_name"].isin(tircp_already_entered)]).reset_index(
    drop=True
)

In [64]:
len(tircp2)

91

In [66]:
# atp_shopp_drmt[atp_shopp_drmt["previous_caltrans_nominations"].str.contains(("tircp"))].drop(columns = "notes")

#### Test with Project Names

In [67]:
# tircp2["project_test"] = tircp2["project_name"]

In [68]:
# atp_shopp_drmt["project_test"] = atp_shopp_drmt["project_name"]

In [69]:
def simplify_project_names(df, column_wanted: str):
    df[column_wanted] = (
        df[column_wanted]
        .str.strip()
        .str.lower()
        .str.replace("/", "")
        .str.replace("-", "")
        .str.replace("!", "")
        .str.replace("&", "")
        .str.replace("#", "")
        .str.replace("(", "")
        .str.replace(")", "")
        .str.replace(":", "")
        .str.replace("the", "")
        .str.strip()  # strip again after getting rid of certain things
    )
    return df

In [70]:
# tircp2 = simplify_project_names(tircp2, "project_test")

In [71]:
# atp_shopp_drmt = simplify_project_names(atp_shopp_drmt, "project_test")

In [72]:
# atp_shopp_drmt[["project_name","project_test"]].sort_values(by = ["project_name"])

In [73]:
# Merge on project names
# test3 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", on=["project_test"], indicator = True, suffixes = ["_shopp", "_tircp"] )

In [74]:
# test3._merge.value_counts()

In [75]:
# atp_shopp_drmt[["project_test"]].sort_values("project_test")

In [76]:
# tircp2[["project_test"]].sort_values("project_test")

#### Test 1 with Fuzzy Matching on Project Names
* Unsuccessful with project names. Only 2 matches and after looking at descriptions/districts manually, they aren't similar enough.

In [77]:
# Replace all rows in agency column with a min ratio with  "string_to_match value"
def replace_matches_in_column(df, column, new_col_name, string_to_match, min_ratio):
    # Get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio
    )

    # Only get matches with a  min ratio
    close_matches = [matches[0] for matches in matches if matches[1] > min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, new_col_name] = string_to_match

In [78]:
# Create a list of all the TIRCP names
# tircp_projects = tircp2.project_name.unique().tolist()

In [79]:
# atp_shopp_projects = atp_shopp_drmt.project_name.unique().tolist()

In [80]:
# for i in atp_shopp_projects:
#    replace_matches_in_column(
#        tircp2, "project_name", "project_name_fuzzy_match", i, 80
#    )

In [81]:
# tircp2[["project_description", "project_description","project_description_fuzzy_match"]]

In [82]:
# Merge on project names
# test4 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", left_on=["project_test"], right_on = ["project_name_fuzzy_match"], indicator = True, suffixes = ["_shopp", "_tircp"] )

In [83]:
# test4._merge.value_counts()

In [84]:
# test4.loc[test4._merge == "both"][["district_shopp","district_tircp","project_name_tircp","project_name_shopp", "project_description_shopp", "project_description_tircp"]]

### Concat & Clean
* After only finding 2 projects that are already entered in, concat TIRCP information. 
    * Already filtered out those 2 projects in TIRCP.
    * Also filtered out any projects where TIRCP amount equals Total Project Cost, since presumbly they don't need any more $.
* Concat with original ATP SHOPP.

In [85]:
atp_drop = ["unnamed:_0", "update_complete",  'merge',
 'detailed_project_title',]

In [86]:
atp_shopp = atp_shopp.drop(columns=atp_drop)

In [87]:
concat1 = pd.concat([atp_shopp, tircp2])

In [88]:
# Fill NA based on dtypes
concat1 = concat1.fillna(concat1.dtypes.replace({"float64": 0.0, "object": "None"}))

In [89]:
# concat1.info(verbose=True)

In [90]:
# concat1.tail(1)

#### Add Detailed District Name

In [91]:
concat1['detailed_project_title'] =  ('District '+ concat1.district.astype('str') + '-' + concat1.project_name)

#### Add Fake Values

In [92]:
# https://stackoverflow.com/questions/64093880/how-to-create-random-floats-and-add-them-as-a-dataframe-column
np.random.seed(365)

In [93]:
fake_columns = [
    "increase_peak_person_throughput",
    "reduction_in_peak_period_delay",
    "reduction_in_fatal_and_injury_crashes",
    "reduction_in_injury_rates",
    "increase_access_to_jobs",
    "increase_access_jobs_to_DAC",
    "commercial_dev_developed",
    "tons_of_goods_impacted",
    "improve_air_quality",
    "impact_natural_resources",
    "support_of_transportation",
]

In [94]:
# Add fake metric columns: I want a random value between 0 to 50.
for i in fake_columns:
    concat1[i] = np.round(np.random.uniform(0.0, 50.0, size=(len(concat1), 1)), 2)

#### Create fake benefit score and ranks.

In [95]:
# Create the fake benefit score based off of Virginia DOT.
concat1["fake_benefit_score"] = (
    (concat1[fake_columns].sum(axis=1))
    / (concat1["total_unfunded_need__$1,000"])
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
    .sum()
) * 1000000

In [96]:
# Create a fake statewide project rank
concat1["statewide_rank"] = concat1["fake_benefit_score"].rank(ascending=False)

In [97]:
# Create fake project rank by district
concat1["district_rank"] = concat1.groupby("district")["fake_benefit_score"].rank(
    method="dense", ascending=False
)

In [98]:
# Assign percentile of project among all projects in the state.
concat1 = _utils.project_size_rating(
    concat1, "statewide_rank", "fake_benefit_score_statewide_percentile"
)

In [99]:
# Make sure this makes sense
# concat1[['district_rank','statewide_rank','fake_benefit_score','fake_benefit_score_statewide_percentile']].sort_values('fake_benefit_score', ascending= False)

In [100]:
# D4 only projects
len(concat1.loc[concat1["district"] == 4])

120

In [101]:
# list(concat1.columns)

#### Add median across districts
Suggestion from Nick to add medians across the district. 
* Project cost
* unfunded needs
* benefit score

In [102]:
# Grab medians
summary_district_state = (
    concat1.groupby(["district"])
    .agg(
        {
            "total_unfunded_need__$1,000": "median",
            "fake_benefit_score": "median",
            "total_project_cost__$1,000": "median",
            "csis_total_score__out_of_45": "median",
            "atp_total_score__out_of_100": "median",
        }
    )
    .reset_index()
)

In [103]:
# Add suffixes
summary_district_state = summary_district_state.add_suffix("_district_median_")

In [104]:
concat1 = pd.merge(
    concat1,
    summary_district_state,
    left_on="district",
    right_on="district_district_median_",
    how="left",
)

#### Clean Up
* Best to look at full district & county names
* Replace county "None" with "full county name" values. 

In [105]:
concat1 = _utils.clean_up_columns(concat1)

In [106]:
len(concat1)

905

In [107]:
concat1['County'] = np.where(concat1['County'] == "None", concat1['Full County Name'], concat1['County'])

In [108]:
tircp3 = concat1.loc[concat1['Previous Caltrans Nominations'] == "TIRCP Cycle 3"]

In [115]:
tircp3.sample()

,Rail Project Id,10 Year Plan,Ct Project Id,Ea,Ppno,Project Name,District,County,Route,Project Description,Current Phase,Con Existing Source S Of Funds,Con Anticipated Source Of Funds,Target Opening Year,Beg Pm,End Pm,Primary Mode,Previous Caltrans Nominations,Urban Rural,Notes,Lead Agency,Pid Approval Date M010,Target Pa Ed M200,Rtl Date M460,Con Start Date M500,Funding Need Phase S,"Pa Ed Cost $1,000","Ps E Cost $1,000","Row Cost $1,000","Con Support Cost $1,000","Non Infrastructure Plan Cost $1,000","Total Unfunded Need $1,000",Previous Funding Request Phase,Last Scored,Csis Alignment,Csis Total Score Out Of 45,Mode Shift Csis Score,Mode Shift Csis Comment,Vmt Csis Score,Vmt Csis Comment,Public Engagement Csis Score,Public Engagement Csis Comment,Dac Local Community Needs Csis Score,Dac Local Community Needs Csis Comment,Safety Csis Score,Safety Csis Comment,Zev Csis Score,Zev Csis Comment,Climate Resiliency Csis Score,Climate Resiliency Csis Comment,Natural Resources And Ecosystems Csis Score,Natural Resources And Ecosystems Csis Comment,Infill Development And Land Use Csis Score,Infill Development And Land Use Csis Comment,Benefits To Dac And Advancing Equity Atp Score,Community Need Atp Score,Safety Atp Score,Public Participation Atp Score,Community Feedback Atp Score,Continued Engagement Atp Score,Context Sensitive And Innovation Atp Score,Transformative Atp Score,Atp Total Score Out Of 100,Atp Alignment,Access Alignment,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,Previous Funding Request,Purpose Need,Parcel Counts,"Total Project Cost $1,000","Con Capital Cost $1,000",Hq Priority,District Priority,Potential Funding Program S,Located In Dac,Shs Capacity Increase Detail,Secondary Mode S,Full County Name,Abbrev,District Full Name,Ppno1,"Total Project Cost $1,000 1","Pa Ed Cost $1,000 1","Ps E Cost $1,000 1","Non Infrastructure Plan Cost $1,000 1",Detailed Project Title,Increase Peak Person Throughput,Reduction In Peak Period Delay,Reduction In Fatal And Injury Crashes,Reduction In Injury Rates,Increase Access To Jobs,Increase Access Jobs To Dac,Commercial Dev Developed,Tons Of Goods Impacted,Improve Air Quality,Impact Natural Resources,Support Of Transportation,Fake Benefit Score,Statewide Rank,District Rank,Fake Benefit Score Statewide Percentile,District District Median,"Total Unfunded Need $1,000 District Median",Fake Benefit Score District Median,"Total Project Cost $1,000 District Median",Csis Total Score Out Of 45 District Median,Atp Total Score Out Of 100 District Median
867,None,0.0,None,None,1155A,Extend rail service to Monterey County,0,Various,None,"This project will implement the extension of two round trip passenger rail services from Gilroy to Salinas, track improvements, positive train control, and a two-train layover facility. There has only been one allocation of $500,000 for Network Integration.",CONST,None,None,None,None,None,Transit/Zev/Rail (Passenger),TIRCP Cycle 3,None,None,Transportation Agency For Monterey County,None,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,71371.0,None,0.0,None,0.0,0.0,None,0.0,None,0.0,None,0.0,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,None,81519.0,0.0,0.0,0.0,None,None,0,None,Various,None,Various,None,0.0,0.0,0.0,0.0,District 0-Extend rail service to Monterey County,31.26,4.83,46.3,41.49,3.07,13.24,30.05,14.85,13.62,0.55,37.9,1.312896,694.0,5.0,75th percentile,0,53739.0,1.623072,87196.969,0.0,0.0


In [111]:
concat1.sample()

,Rail Project Id,10 Year Plan,Ct Project Id,Ea,Ppno,Project Name,District,County,Route,Project Description,Current Phase,Con Existing Source S Of Funds,Con Anticipated Source Of Funds,Target Opening Year,Beg Pm,End Pm,Primary Mode,Previous Caltrans Nominations,Urban Rural,Notes,Lead Agency,Pid Approval Date M010,Target Pa Ed M200,Rtl Date M460,Con Start Date M500,Funding Need Phase S,"Pa Ed Cost $1,000","Ps E Cost $1,000","Row Cost $1,000","Con Support Cost $1,000","Non Infrastructure Plan Cost $1,000","Total Unfunded Need $1,000",Previous Funding Request Phase,Last Scored,Csis Alignment,Csis Total Score Out Of 45,Mode Shift Csis Score,Mode Shift Csis Comment,Vmt Csis Score,Vmt Csis Comment,Public Engagement Csis Score,Public Engagement Csis Comment,Dac Local Community Needs Csis Score,Dac Local Community Needs Csis Comment,Safety Csis Score,Safety Csis Comment,Zev Csis Score,Zev Csis Comment,Climate Resiliency Csis Score,Climate Resiliency Csis Comment,Natural Resources And Ecosystems Csis Score,Natural Resources And Ecosystems Csis Comment,Infill Development And Land Use Csis Score,Infill Development And Land Use Csis Comment,Benefits To Dac And Advancing Equity Atp Score,Community Need Atp Score,Safety Atp Score,Public Participation Atp Score,Community Feedback Atp Score,Continued Engagement Atp Score,Context Sensitive And Innovation Atp Score,Transformative Atp Score,Atp Total Score Out Of 100,Atp Alignment,Access Alignment,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,Previous Funding Request,Purpose Need,Parcel Counts,"Total Project Cost $1,000","Con Capital Cost $1,000",Hq Priority,District Priority,Potential Funding Program S,Located In Dac,Shs Capacity Increase Detail,Secondary Mode S,Full County Name,Abbrev,District Full Name,Ppno1,"Total Project Cost $1,000 1","Pa Ed Cost $1,000 1","Ps E Cost $1,000 1","Non Infrastructure Plan Cost $1,000 1",Detailed Project Title,Increase Peak Person Throughput,Reduction In Peak Period Delay,Reduction In Fatal And Injury Crashes,Reduction In Injury Rates,Increase Access To Jobs,Increase Access Jobs To Dac,Commercial Dev Developed,Tons Of Goods Impacted,Improve Air Quality,Impact Natural Resources,Support Of Transportation,Fake Benefit Score,Statewide Rank,District Rank,Fake Benefit Score Statewide Percentile,District District Median,"Total Unfunded Need $1,000 District Median",Fake Benefit Score District Median,"Total Project Cost $1,000 District Median",Csis Total Score Out Of 45 District Median,Atp Total Score Out Of 100 District Median
846,None,0.0,None,None,CP029,Los Angeles City: Leading the Transformation to Zero-Emission Electric Bus Transit Service,7,Los Angeles,None,"Acquire 112 zero-emission buses to replace existing propane vehicles and add new vehicles, in order to increase frequency of all existing DASH routes to 15-minute service and add 4 new routes, serving communities throughout the City of Los Angeles as recommended in the comprehensive Transit Service Analysis.",CONST,None,None,None,None,None,Transit/Zev/Rail (Passenger),TIRCP Cycle 3,None,None,City Of Los Angeles,None,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,66686.0,None,0.0,None,0.0,0.0,None,0.0,None,0.0,None,0.0,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,None,102790.0,0.0,0.0,0.0,None,None,0,None,Los Angeles,None,District 7: Los Angeles,None,0.0,0.0,0.0,0.0,District 7-Los Angeles City: Leading the Transformation to Zero-Emission Electric Bus Transit Service,21.64,14.13,2.21,23.05,24.64,45.63,9.36,37.88,48.37,27.53,19.38,1.515842,450.0,61.0,50th percentile,7,0.0,1.501587,20500.0,0.0,0.0
